# Part 6.1 - Prepare Dataframe
---

### Papers Past Topic Modeling
<br/>


Ben Faulks - bmf43@uclive.ac.nz

Xiandong Cai - xca24@uclive.ac.nz

Yujie Cui - ycu23@uclive.ac.nz

In [1]:
import sys
sys.path.insert(0, '../utils')
import pandas as pd
pd.set_option('display.max_columns', 120)
pd.set_option('display.max_colwidth', 120)
from pyspark.sql import functions as F
from pyspark.sql.types import *
from utils import conf_pyspark, load_dataset
from utils_preplot import preplot

# intiate PySpark
sc, spark = conf_pyspark()

sc

[('spark.driver.port', '32888'),
 ('spark.driver.host', 'x99.hub'),
 ('spark.app.name', 'local'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.memory', '62g'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1548324236019'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.cores', '6'),
 ('spark.driver.maxResultSize', '4g')]


<SparkContext master=local[*] appName=local>

**This part we generate the dataframe that part 6.2 will be used:**
* dominant topic per year dataframe;
* average topic weight per year dataframe;
* dominant topic per month dataframe (only for WWI analysis);
* average topic weight per month dataframe (only for WWI analysis);

**After training topic models, we got the topic words list (**`topicKeys.txt`**) and the doc-topic matrix (**`docTopics.txt`**). The doc-topic matrix is a docmument based file which means we could connect the topic weights in the matrix and the metadata (date, region, publisher) in the meta dataset, then the combined dataset could apply to accomplish many data mining or statistical works, for instance, the granularity of time can achieve 1 day, which could perform time series analsys. Now we load those data to generate dataframe for analysys and visualization.**

## 1 Prepare Dataframe for Train Set

### 1.1 Load data

**Load metadata ("id", "region" and "date"):**

In [2]:
df_meta = load_dataset('meta', spark).select(F.col('id').alias('id_'), F.col('region'), F.col('date'))

In [3]:
df_meta.limit(5).toPandas().head()

,id_,region,date
0,1854215,Auckland,1862-06-14
1,1854233,Auckland,1863-08-01
2,1854245,Auckland,1863-09-01
3,1854253,Auckland,1863-09-01
4,1854264,Auckland,1863-10-01


**Load topic words list:**

In [4]:
path = r'../models/train/topicKeys.txt'

data_schema = StructType([
    StructField('topic', IntegerType()),
    StructField('weight_', FloatType()),
    StructField('words', StringType())
])

df_topics = (
    spark.read.format("com.databricks.spark.csv")
    .option("header", "false")
    .option("inferSchema", "false")
    .option("delimiter", "\t")
    .schema(data_schema)
    .load(path)
)
topic_number = df_topics.count()

In [5]:
print('Shape of dataframe: ({}, {})'.format(topic_number, len(df_topics.columns)))
df_topics.limit(10).toPandas().head(10)

Shape of dataframe: (200, 3)


,topic,weight_,words
0,0,0.00521,sale auction land property section lot sell sections acres public block instructions estate street auctioneers front...
1,1,0.00335,printing prizes cards prize horse tickets office drawing books divided forms programmes advertisements stamps column...
2,2,0.00631,seeds seed grass manure clover potatoes special manures oats turnip white red cocksfoot rape ton garden sacks stock ...
3,3,0.00710,company miss theatre opera night evening house performance audience stage play season part comedy success great ente...
4,4,0.01504,man life men people great work good world time young make things fact human mind women nature years knowledge social
5,5,0.00429,fish island islands fishing south north sea trout river line bay west east boundary coast salmon point southern wate...
6,6,0.01112,evening meeting held yesterday messrs morning night saturday number place local monday made large afternoon present ...
7,7,0.00748,price ladies prices silk special white costumes smart black latest coats yard wear blouses styles shades advertiseme...
8,8,0.01205,cough advertisements column colds cure remedy cold coughs advt bottle throat great woods lung peppermint nazol baxte...
9,9,0.00902,bank cent loan new_zealand money london interest banks rate deposits exchange colonial stock bills notes debentures ...


**Load doc-topic matrix:**

In [6]:
path = r'../models/train/docTopics.txt'

# generate new column names
columns = [str(x) for x in list(range(topic_number))]
columns.insert(0, 'id')
columns.insert(0, 'index')

# load data
df_doctopic = (
    spark.read.format("com.databricks.spark.csv")
    .option("header", "false")
    .option("inferSchema", "true")
    .option("delimiter", "\t")
    .load(path)
)

# change columns name and drop # column which is table index and useless
df_doctopic = df_doctopic.toDF(*columns)

In [7]:
print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

df_doctopic.limit(5).toPandas().head()

Shape of dataframe: (603629, 202)


,index,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,...,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
0,0,1854215,0.000089,0.000057,0.000108,0.000121,0.000256,0.000073,0.000190,0.000128,0.000205,0.000154,0.000274,0.000114,0.000087,0.000123,0.000108,0.000092,0.000084,0.000048,0.000032,0.000187,0.000063,0.000063,0.000143,0.000172,0.000113,0.000131,0.000176,0.000032,0.000083,0.000103,0.000163,0.000088,0.000139,0.000250,0.000043,0.000230,0.000135,0.000272,0.000146,0.000164,0.051214,0.000030,0.000155,0.000030,0.000204,0.000657,0.000182,0.000115,0.000198,0.000154,0.000190,0.000034,0.000118,0.000088,0.000263,0.000202,0.000154,0.000029,...,0.000220,0.000164,0.000074,0.000156,0.000170,0.000124,0.000224,0.000099,0.000089,0.000511,0.000179,0.000087,0.000341,0.000094,0.000060,0.000131,0.000101,0.000117,0.000073,0.000146,0.000104,0.000110,0.000095,0.000120,0.000136,0.000198,0.000125,0.000311,0.000228,0.000083,0.000105,0.000130,0.000181,0.000184,0.000282,0.000112,0.375211,0.000062,0.000229,0.000122,0.000067,0.000111,0.000124,0.000107,0.017285,0.000330,0.000152,0.000201,0.000106,0.000140,0.000153,0.000125,0.000072,0.000122,0.000073,0.000082,0.000116,0.000035,0.000076,0.000186
1,1,1854233,0.000036,0.000023,0.000043,0.000048,0.000103,0.000029,0.000076,0.000051,0.000082,0.000062,0.000110,0.000046,0.000035,0.000049,0.000043,0.000037,0.000033,0.000019,0.000013,0.000075,0.000025,0.000025,0.000057,0.000069,0.000045,0.000052,0.000070,0.000013,0.000033,0.000041,0.000065,0.000035,0.000056,0.000100,0.000017,0.000092,0.000054,0.000109,0.000058,0.000066,0.000028,0.000012,0.000062,0.000012,0.000082,0.000263,0.156899,0.000046,0.047809,0.000062,0.000076,0.000014,0.000047,0.000035,0.000105,0.000081,0.000061,0.000012,...,0.000088,0.000066,0.000030,0.088703,0.000068,0.000050,0.000089,0.000040,0.000036,0.000204,0.000071,0.000035,0.000136,0.000037,0.000024,0.000053,0.000040,0.000047,0.000029,0.000058,0.000042,0.000044,0.000038,0.000048,0.000054,0.000079,0.000050,0.395599,0.000091,0.000033,0.000042,0.000052,0.000073,0.000073,0.000113,0.000045,0.000066,0.000025,0.000092,0.000049,0.000027,0.000044,0.000050,0.000043,0.000095,0.000132,0.000061,0.000080,0.000042,0.000056,0.000061,0.000050,0.000029,0.000049,0.000029,0.000033,0.000047,0.000014,0.000031,0.000074
2,2,1854245,0.000032,0.000021,0.000039,0.000044,0.000092,0.000026,0.000068,0.000046,0.000074,0.000055,0.000099,0.000041,0.000031,0.000044,0.000039,0.000033,0.000030,0.000017,0.000011,0.000067,0.000023,0.000023,0.000052,0.000062,0.000041,0.000047,0.000063,0.000011,0.000030,0.000037,0.000059,0.000032,0.000050,0.000090,0.000016,0.000083,0.000049,0.000098,0.000053,0.000059,0.000026,0.000011,0.000056,0.000011,0.000074,0.000237,0.000065,0.000041,0.000071,0.000056,0.036956,0.000012,0.006190,0.000032,0.123051,0.000073,0.000055,0.000010,...,0.000079,0.000059,0.000027,0.000056,0.000061,0.000045,0.000081,0.000036,0.000032,0.012480,0.000064,0.000032,0.000123,0.000034,0.000021,0.000047,0.000036,0.000042,0.000026,0.000053,0.000037,0.000040,0.000034,0.000043,0.000049,0.000072,0.000045,0.135364,0.000082,0.000030,0.000038,0.000047,0.000065,0.000066,0.000102,0.000040,0.104572,0.000022,0.000083,0.000044,0.000024,0.000040,0.000045,0.000039,0.000086,0.000119,0.000055,0.000072,0.000038,0.000050,0.000055,0.000045,0.000026,0.000044,0.000026,0.000029,0.000042,0.000013,0.000028,0.000067
3,3,1854253,0.075759,0.000008,0.000014,0.000016,0.000035,0.000010,0.000026,0.000017,0.000028,0.000021,0.000037,0.000015,0.000012,0.000017,0.280050,0.000012,0.000011,0.000006,0.000004,0.000025,0.000008,0.000009,0.000019,0.000023,0.000015,0.000018,0.000024,0.000004,0.000011,0.000014,0.000022,0.000012,0.000019,0.000034,0.000006,0.000031,0.000018,0.0

**In above dataframe, "index" column is the row number, "id" column is the sample/document/text id, the same with "id" in dataset, "0" to max topic number columns are the weight of each topic per document.**

### 1.2 Add Dominant Topics Column

**Find dominant topic of each document:**

In [8]:
# https://stackoverflow.com/questions/46819405/how-to-get-the-name-of-column-with-maximum-value-in-pyspark-dataframe

def argmax(cols, *args):
    return [c for c, v in zip(cols, args) if v == max(args)][0]

def add_domtopic(df):
    """
    find the dominant topic of each sample/row/document
    input: dataframe of weight of each topic
    output: the dominant topic number dataframe
    """
    argmax_udf = lambda cols: F.udf(lambda *args: argmax(cols, *args), StringType())
    return (df
            .withColumn('domtopic',
                        argmax_udf(df.columns[2:])(*df.columns[2:]))
            .withColumn('weight', 
                        F.greatest(*[F.col(x) for x in df.columns[2:-1]])))

# add the df_dominant to doc-topic matrix
df_doctopic = add_domtopic(df_doctopic)

In [9]:
#print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

#df_doctopic.limit(5).toPandas().head()

### 1.3 Add Metadata Columns

**Here we only add "region" and "date" column as metadata, the accuracy of time could achieve "day" level, using the time series features we could convert "date" to year base or month base etc. depending on the need.**

In [10]:
df_doctopic = (df_doctopic
               .join(df_meta, df_doctopic.id == df_meta.id_)
               .withColumn('year', F.date_format('date', 'yyyy'))
               .drop('id_')
               .drop('date')
               .orderBy('index'))

In [11]:
#print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

#df_doctopic.limit(5).toPandas().head()

### 1.4 Document - Dominant Topics Dataframe

**It is hard to plot the doc-topic matrix (high dimension), we need transform it to extract or reduce features. First we generate dominant topics dataframe which could be used to reveal the relationship between dominant topics and region/year.**

In [12]:
df_docdomtopic = (df_doctopic
                  .join(df_topics, df_doctopic.domtopic == df_topics.topic)
                  .select(F.col('id'), 
                          F.col('region'), 
                          F.col('year'), 
                          F.col('domtopic'), 
                          F.col('weight'), 
                          F.col('words'))
                  .orderBy('id'))

df_docdomtopic.cache()

DataFrame[id: int, region: string, year: string, domtopic: string, weight: double, words: string]

In [13]:
print('Shape of dataframe: ({}, {})'.format(df_docdomtopic.count(), len(df_docdomtopic.columns)))

df_docdomtopic.limit(5).toPandas().head()

Shape of dataframe: (603629, 6)


,id,region,year,domtopic,weight,words
0,1854215,Auckland,1862,176,0.375211,advertisements office paper published column post times news notice new_zealand subscribers advertising insertion pu...
1,1854233,Auckland,1863,167,0.395599,business public orders street notice general begs stock attention premises made inform advertisements goods prices p...
2,1854245,Auckland,1863,76,0.276675,apply freight passage passengers new_zealand steam company wellington london agents tons shipping leave melbourne li...
3,1854253,Auckland,1863,14,0.280050,act person regulations made time persons section part exceeding land provisions law provided liable property payment...
4,1854264,Auckland,1863,188,0.263871,estate creditors public trustee bankruptcy day notice office trust bankrupt court accounts paid business debts act t...


**Save the dataframe for later use:**

In [14]:
path = r'../models/train/domTopics'

df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist()

DataFrame[id: int, region: string, year: string, domtopic: string, weight: double, words: string]

In [15]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

### 1.5 Average Weight Topics Dataframe

**Beside dominant topics dataframe, we could calculate average weight of each topic in a year and create dataframe for it, which could reveal the weight variety of each topic as time goes on, we could take the average weight as features and execute data mining algorithms to find patterns, e.g. correlation between features. The weights of each topic were already scaled to 0-1 by default, so the sum of average weight of each year is 1, we do not need to scale it.**

In [16]:
df_avgweight = (df_doctopic
                .drop('index')
                .drop('id')
                .drop('domtopic')
                .drop('region')
                .drop('weight')
                .groupBy('year')
                .avg()
                .orderBy('year'))

In [17]:
df_avgweight.limit(5).toPandas().head()

,year,avg(0),avg(1),avg(2),avg(3),avg(4),avg(5),avg(6),avg(7),avg(8),avg(9),avg(10),avg(11),avg(12),avg(13),avg(14),avg(15),avg(16),avg(17),avg(18),avg(19),avg(20),avg(21),avg(22),avg(23),avg(24),avg(25),avg(26),avg(27),avg(28),avg(29),avg(30),avg(31),avg(32),avg(33),avg(34),avg(35),avg(36),avg(37),avg(38),avg(39),avg(40),avg(41),avg(42),avg(43),avg(44),avg(45),avg(46),avg(47),avg(48),avg(49),avg(50),avg(51),avg(52),avg(53),avg(54),avg(55),avg(56),avg(57),avg(58),...,avg(140),avg(141),avg(142),avg(143),avg(144),avg(145),avg(146),avg(147),avg(148),avg(149),avg(150),avg(151),avg(152),avg(153),avg(154),avg(155),avg(156),avg(157),avg(158),avg(159),avg(160),avg(161),avg(162),avg(163),avg(164),avg(165),avg(166),avg(167),avg(168),avg(169),avg(170),avg(171),avg(172),avg(173),avg(174),avg(175),avg(176),avg(177),avg(178),avg(179),avg(180),avg(181),avg(182),avg(183),avg(184),avg(185),avg(186),avg(187),avg(188),avg(189),avg(190),avg(191),avg(192),avg(193),avg(194),avg(195),avg(196),avg(197),avg(198),avg(199)
0,1839,0.000018,0.000011,0.000021,0.000024,0.000051,0.000015,0.000038,0.000025,0.000041,0.000031,0.000054,0.000023,0.000017,0.000024,0.000021,0.000018,0.000017,0.000009,0.000006,0.000037,0.000012,0.000013,0.000028,0.000034,0.000022,0.000026,0.000035,0.000006,0.000016,0.000020,0.000032,0.000017,0.000028,0.000050,0.000009,0.000046,0.000027,0.000054,0.000029,0.000033,0.000014,0.000006,0.000031,0.000006,0.050775,0.000130,0.000036,0.000023,0.121801,0.000031,0.000038,0.000007,0.000023,0.074427,0.125196,0.000040,0.000030,0.000006,0.000027,...,0.000044,0.000033,0.000015,0.000031,0.000034,0.000025,0.000044,0.000020,0.000018,0.000101,0.000035,0.000017,0.050802,0.000019,0.000012,0.000026,0.000020,0.000023,0.000014,0.000029,0.000021,0.000022,0.000019,0.000024,0.000027,0.000039,0.057523,0.091383,0.000045,0.000017,0.000021,0.000026,0.000036,0.000036,0.000056,0.000022,0.000033,0.000012,0.000045,0.000024,0.000013,0.000022,0.000025,0.108254,0.000047,0.000065,0.000030,0.054156,0.000021,0.000028,0.000030,0.000025,0.000014,0.000024,0.000014,0.000016,0.000023,0.000007,0.000015,0.000037
1,1840,0.009794,0.000353,0.000964,0.000191,0.000970,0.007205,0.000816,0.000654,0.000164,0.009952,0.006066,0.001648,0.000069,0.000098,0.000086,0.000073,0.008620,0.000917,0.001717,0.000506,0.000050,0.000050,0.005598,0.000137,0.000090,0.000104,0.000235,0.005870,0.000066,0.008919,0.000130,0.001495,0.000111,0.000199,0.001922,0.161458,0.001473,0.000217,0.001729,0.001743,0.001162,0.000024,0.000123,0.000024,0.012510,0.003926,0.000452,0.000092,0.007054,0.000123,0.000591,0.000666,0.008032,0.000070,0.048430,0.001878,0.000374,0.000023,0.012114,...,0.000175,0.002585,0.000059,0.006099,0.000135,0.024421,0.000178,0.000079,0.000071,0.001695,0.000142,0.000070,0.014149,0.000075,0.000047,0.000493,0.017409,0.014699,0.116515,0.004178,0.028201,0.000088,0.000076,0.000096,0.023714,0.000158,0.019909,0.026838,0.000181,0.000066,0.000084,0.005239,0.000144,0.000536,0.000450,0.000089,0.036599,0.000049,0.000540,0.000097,0.001580,0.002613,0.000099,0.000085,0.000441,0.007029,0.000121,0.001053,0.011050,0.002704,0.000122,0.000100,0.000058,0.000097,0.000058,0.000065,0.000093,0.000028,0.000061,0.000148
2,1841,0.013980,0.000038,0.000072,0.000176,0.000171,0.000341,0.000852,0.000085,0.000137,0.011508,0.000646,0.000076,0.000058,0.000082,0.008961,0.000061,0.017418,0.000032,0.000021,0.000318,0.000042,0.000042,0.002624,0.000114,0.000076,0.002187,0.012274,0.000021,0.000055,0.000069,0.000109,0.000269,0.000093,0.000268,0.000448,0.000878,0.000602,0.015167,0.000413,0.003102,0.000047,0.000020,0.000103,0.000604,0.026763,0.003836,0.001440,0.000077,0.022351,0.000103,0.000127,0.000023,0.006397,0.000059,0.059771,0.001405,0.000273,0.000388,0.000092,...,0.018402,0.005332,0.005324,0.000104,0.000113,0.002661,0.007749,0.000861,0.000594,0.002558,0.000119,0.000440,0.045838,0.000062,0.011170,0.000190,0.000067,0.013064,0.052197,0.001411,0.009299,0.007965,0.000262,0.000949,0.000663,0.000132,0.058922,0.051073,0.006915,0.000056,0.000070,0.0

**check years are identical with dataset:**

In [18]:
print('Year range of average weight dataframe:', 
      df_avgweight.select(F.min('year'), F.max('year')).first(), 
      '\nYear number:',
      df_avgweight.select('year').distinct().count())
print('Year range of doc-topic      dataframe:', 
      df_doctopic.select(F.min('year'), F.max('year')).first(), 
      '\nYear number:',
      df_doctopic.select('year').distinct().count())

Year range of average weight dataframe: Row(min(year)='1839', max(year)='1945') 
Year number: 104
Year range of doc-topic      dataframe: Row(min(year)='1839', max(year)='1945') 
Year number: 104


**The dimension of the avrage weight dataframe is small (topic_n\*year_n), so we directly save the dataframe using Pandas:**

In [19]:
path = r'../models/train/avgWeight.csv'

df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

## 2 Prepare Dataframe for Subset

**The process for subset is the same with for full dataset, we wrapped the process to a function and call it for each subset.**

### 2.1 By Range of Time

**Generate dataframes:**

In [20]:
file = r'../models/wwi/docTopicsInfer.txt'

df_docdomtopic, df_avgweight = preplot(file, df_meta, df_topics, 'month', spark)

**Save dataframes:**

In [21]:
path = r'../models/wwi/avgWeight.csv'
df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

path = r'../models/wwi/domTopics/'
df_docdomtopic.write.csv(path, mode='overwrite')

**Convert multi files to a csv file:**

In [22]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

### 2.2 By Region

**Generate dataframes:**

In [23]:
file = r'../models/regions/docTopicsInfer.txt'

df_docdomtopic, df_avgweight = preplot(file, df_meta, df_topics, 'year', spark)

**Save dataframes:**

In [24]:
path = r'../models/regions/avgWeight.csv'
df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

path = r'../models/regions/domTopics/'
df_docdomtopic.write.csv(path, mode='overwrite')

**Convert multi files to a csv file:**

In [25]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

### 2.3 By Label

**Generate dataframes:**

In [26]:
file = r'../models/ads/docTopicsInfer.txt'

df_docdomtopic, df_avgweight = preplot(file, df_meta, df_topics, 'year', spark)

**Save dataframes:**

In [27]:
path = r'../models/ads/avgWeight.csv'
df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

path = r'../models/ads/domTopics'
df_docdomtopic.write.csv(path, mode='overwrite')

**Convert multi files to a csv file:**

In [28]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

---